In [1]:
import math
import pandas as pd
import numpy as np
from scipy.stats import entropy
from numpy.linalg import norm
import math


In [2]:
user_reading_history = pd.read_csv("goodbook/trainVal-book0.csv", sep=",", names= ["user", "item", "rating", "genre"])
user_reading_history

,user,item,rating,genre
0,1,3,2.0,"classics, science, fiction, fantasy, philosoph..."
1,1,4,5.0,"science, ebooks, religion, philosophy, classic..."
2,1,5,4.0,"ebooks, fiction, classics, contemporary, roman..."
3,1,7,4.0,"religion, ebooks, spirituality, romance, philo..."
4,1,8,5.0,"romance, contemporary, fiction, classics"
...,...,...,...,...
8472,943,64,5.0,"science, fiction, classics, philosophy, busine..."
8473,943,90,5.0,"memoir, classics, nonfiction, travel, history,..."
8474,943,19,5.0,"music, fiction, classics, ebooks, history, con..."
8475,943,109,5.0,"classics, ebooks, thriller, mystery, fiction, ..."


In [3]:

class ShannonEntropy:
    _ln2 = math.log(2.0)  # Define _ln2 as a class attribute

    def __init__(self, user_recommendations, user_relevant_items):
        self.user_recommendations = user_recommendations
        self.user_relevant_items = user_relevant_items

    @staticmethod
    def name():
        return "SEntropy"

    def _normalize_scores(self):
        scores = [rec[1] for user_rec in self.user_recommendations.values() for rec in user_rec]
        min_score = min(scores)
        max_score = max(scores)
        self._min_max_range = max_score - min_score  # Store the range for later use

        # Perform Min-Max scaling
        for user, user_rec in self.user_recommendations.items():
            for i, (item, score) in enumerate(user_rec):
                normalized_score = (score - min_score) / self._min_max_range
                self.user_recommendations[user][i] = (item, normalized_score)

    def __user_se(self, user_recommendations, cutoff):
        user_norm = len(user_recommendations[:cutoff])
        self._free_norm += user_norm
        for i, _ in user_recommendations[:cutoff]:
            self._item_count[i] = self._item_count.get(i, 0) + 1
            self._item_weights[i] = self._item_weights.get(i, 0) + (1 / user_norm)

    def __sales_novelty(self, i):
        return -math.log(self._item_count[i] / self._free_norm) / self._ln2

    def eval(self, cutoff=5):
        self._item_count = {}
        self._item_weights = {}
        self._free_norm = 0

        # Normalize scores
        self._normalize_scores()

        for user, recommendations in self.user_recommendations.items():
            user_recommendations_list = recommendations[:cutoff]
            self.__user_se(user_recommendations_list, cutoff)

        return sum([w * self.__sales_novelty(i) for i, w in self._item_weights.items()])/len(self.user_recommendations)


In [4]:
# Load your datasets
user_recommendations_df = pd.read_csv("goodbook/results/RS-5_0.0_Original.csv")
user_relevant_items_df = pd.read_csv("goodbook/trainVal-book0.csv", sep=",", names= ["user", "item", "rating", "genre"])


FileNotFoundError: [Errno 2] No such file or directory: 'goodbook/results/RS-5_0.0_Original.csv'

In [16]:
# Adapt the dataframes to fit the function's parameters
user_recommendations = user_recommendations_df.groupby('user')[['item', 'score']].apply(lambda x: x.values.tolist()).to_dict()
user_relevant_items = user_relevant_items_df.groupby('user')['item'].apply(list).to_dict()

# Initialize the ShannonEntropy class with the adapted datasets
entropy_metric = ShannonEntropy(user_recommendations, user_relevant_items)

# Evaluate the metric
result = entropy_metric.eval()

print("Shannon Entropy:", result)

Shannon Entropy: 7.9611163528376885


In [152]:
def opt_merge_max_mappings(dict1, dict2):
    """ Merges two dictionaries based on the largest value in a given mapping.
    Parameters
    ----------
    dict1 : Dict[Any, Comparable]
    dict2 : Dict[Any, Comparable]
    Returns
    -------
    Dict[Any, Comparable]
        The merged dictionary
    """
    # we will iterate over `other` to populate `merged`
    merged, other = (dict1, dict2) if len(dict1) > len(dict2) else (dict2, dict1)
    merged = dict(merged)

    for key in other:
        if key not in merged or other[key] > merged[key]:
            merged[key] = other[key]
    return merged


def KL_symmetric(a, b):
    return (entropy(a, b, base=2) + entropy(b, a, base=2)) / 2


def JSD(P, Q):
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    try:
        jsd_root = math.sqrt(abs(0.5 * (entropy(_P, _M, base=2) + entropy(_Q, _M, base=2))))
    except ZeroDivisionError:
        print(P)
        print(Q)
        print()
        jsd_root = None
    return jsd_root

class Calibration:

    def __init__(self, config):
        self.language = config['language']

    def compute_distr(self, recommendations, genre_data):
        id_to_genre = {row['item']: row['genre'] for index, row in genre_data.iterrows()}

        distr = {}
        for item_id, genres_list in recommendations.items():
            for item in genres_list:
                genres = id_to_genre.get(item, '')
                for genre in genres.split(','):
                    genre = genre.strip()
                    category_freq = distr.get(genre, 0)
                    distr[genre] = category_freq + 1
        total_recommendations = sum(distr.values())
        distr = {category: count / total_recommendations for category, count in distr.items()}
        return distr
    
    def compute_kl_divergence(self, p, q):
        # Combine keys from both dictionaries
        all_keys = set(p.keys()).union(q.keys())
        
        # Initialize arrays for frequency values
        p_values = np.zeros(len(all_keys))
        q_values = np.zeros(len(all_keys))
        
        # Populate arrays with frequency values
        for i, key in enumerate(all_keys):
            p_values[i] = p.get(key, 0)
            q_values[i] = q.get(key, 0)
        
        # Normalize distributions
        p_values /= np.sum(p_values)
        q_values /= np.sum(q_values)
        
        # Compute KL divergence
        kl_div = entropy(p_values, q_values)
        js_div = JSD(p_values, q_values)
        
        return [kl_div, js_div]

    
    # def compute_kl_divergence(self, p, q):
    #     return entropy(p.values(), q.values())
    
    def category_divergence(self, reading_history, recommendation, categories):
        freq_rec = self.compute_distr(recommendation, categories)
        freq_history = self.compute_distr(reading_history, categories)
        # print("Frequency distribution of reading history:", freq_history)
        # print("Frequency distribution of recommendation:", freq_rec)
        divergence = self.compute_kl_divergence(freq_history, freq_rec)  
        return divergence

    def calculate_user_level(self, user_reading_history, user_recommendation, genre_data, num_iterations=1):
        kl_divergences = {}
        js_divergences = {}
        
        for algo in user_recommendation['Algorithm'].unique():
            # Filter user_recommendation DataFrame for the current Algorithm
            user_recommendation_algo = user_recommendation[user_recommendation['Algorithm'] == algo]
            
            kl_divergences[algo] = []
            js_divergences[algo] = []
            
            for _ in range(num_iterations):
                kl, js = self.category_divergence(user_reading_history, user_recommendation_algo, genre_data)
                kl_divergences[algo].append(kl)
                js_divergences[algo].append(js)
        
        avg_kl_divergences = {algo: np.mean(divergences) for algo, divergences in kl_divergences.items()}
        avg_js_divergences = {algo: np.mean(divergences) for algo, divergences in js_divergences.items()}
        
        return avg_kl_divergences, avg_js_divergences


In [162]:
# user_recommendation = pd.read_csv("goodbook/results/RS-KNN-10_0.05_2Step.csv")
p_obfuscation = 0.05
preprocess = ["Add", "2Step", "original"][1] # "1Step", "Obf", "original"
topk = [5, 10][1] #
# train = pd.read_csv ("goodbook/trainVal-book0.csv", sep=",", names= ["user", "item", "rating", "genre"]) # 
user_reading_history = pd.read_csv (f"goodbook/Adding_user_item_matrix_{p_obfuscation}_top50Inditems_top100IndiUsers_Categories.csv", sep=",", names= ["user", "item"])#, "rating", "genre"]) # 
user_recommendation = pd.read_csv(f"goodbook/results/RS-KNN-{topk}_{p_obfuscation}_{preprocess}.csv")


genre_data = pd.read_csv("/Users/mslokom/Documents/RecSys_News/goodbook/available_genres_books.csv")
genre_data= genre_data.rename(columns={'book_id': 'item'})

In [163]:
user_reading_history

,user,item
0,1,3
1,1,4
2,1,5
3,1,7
4,1,8
...,...,...
9665,943,109
9666,943,127
9667,943,204
9668,943,381


In [156]:
config = {'language': 'english'}

calibration = Calibration(config)
user_id = 1  # Example user ID
user_reading_history_for_user = user_reading_history[user_reading_history['user'] == user_id]

calibration_scores = calibration.calculate_user_level(user_reading_history_for_user, user_recommendation, genre_data)
print("Average KL Divergence:", calibration_scores[0])
print("Average JS Divergence:", calibration_scores[1])


Average KL Divergence: {'ItemKNN': 1.300323624735443, 'UserKNN': 1.2568280575649549}
Average JS Divergence: {'ItemKNN': 0.6501563710173643, 'UserKNN': 0.6423915484567495}


In [164]:
config = {'language': 'english'}

calibration = Calibration(config)

results = pd.DataFrame(columns=['user', 'Algorithm', 'KL_div', 'JS_div'])

# Iterate over each unique user ID
for user_id in user_reading_history['user'].unique():
    print (user_id)
    # Filter user reading history for the current user
    user_reading_history_for_user = user_reading_history[user_reading_history['user'] == user_id]
    
    # Filter user recommendation for the current user
    user_recommendation_for_user = user_recommendation[user_recommendation['user'] == user_id]
    
    # Iterate over each unique algorithm
    for algo in user_recommendation_for_user['Algorithm'].unique():
        # Filter user recommendation for the current algorithm
        user_recommendation_for_algo = user_recommendation_for_user[user_recommendation_for_user['Algorithm'] == algo]
        
        # Calculate the calibration scores for the current user and algorithm
        calibration_scores = calibration.calculate_user_level(user_reading_history_for_user, user_recommendation_for_algo, genre_data)
        
        # Append the results to the DataFrame
        results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)

        # Print the user-level calibration scores
        # print("User ID:", user_id, "Algorithm:", algo)
        # print("Average KL Divergence:", calibration_scores[0])
        # print("Average JS Divergence:", calibration_scores[1])

# Save the results to a CSV file
# results.to_csv('goodbook/results/book_calibration_results_original.csv', index=False)
results.to_csv(f'goodbook/results/book_calibration_results-KNN-{topk}_{p_obfuscation}_{preprocess}.csv', index=False)



1


/var/folders/8v/cgqpzdls33l6jdsvj3z4fdyh0000gn/T/ipykernel_64710/506424766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)
/var/folders/8v/cgqpzdls33l6jdsvj3z4fdyh0000gn/T/ipykernel_64710/506424766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)


2


/var/folders/8v/cgqpzdls33l6jdsvj3z4fdyh0000gn/T/ipykernel_64710/506424766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)
/var/folders/8v/cgqpzdls33l6jdsvj3z4fdyh0000gn/T/ipykernel_64710/506424766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)


3


...

/var/folders/8v/cgqpzdls33l6jdsvj3z4fdyh0000gn/T/ipykernel_64710/506424766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)
/var/folders/8v/cgqpzdls33l6jdsvj3z4fdyh0000gn/T/ipykernel_64710/506424766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)


942


/var/folders/8v/cgqpzdls33l6jdsvj3z4fdyh0000gn/T/ipykernel_64710/506424766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)
/var/folders/8v/cgqpzdls33l6jdsvj3z4fdyh0000gn/T/ipykernel_64710/506424766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)


943


/var/folders/8v/cgqpzdls33l6jdsvj3z4fdyh0000gn/T/ipykernel_64710/506424766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)
/var/folders/8v/cgqpzdls33l6jdsvj3z4fdyh0000gn/T/ipykernel_64710/506424766.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'user': user_id, 'Algorithm': algo, 'KL_div': calibration_scores[0], 'JS_div': calibration_scores[1]}, ignore_index=True)


In [180]:
# # Load the results DataFrame
p_obfuscation = 0.05
preprocess = ["Add", "Obf", "original"][1] # "1Step", "2Step", "original"
topk = [5, 10][1] # 
# results = pd.read_csv(f'goodbook/results/book_calibration_results-{topk}_{p_obfuscation}_{preprocess}.csv') # book_calibration_results_original
results = pd.read_csv("goodbook/results/book_calibration_results-KNN-10_0.05_2Step.csv")
# # train = pd.read_csv ("goodbook/trainVal-book0.csv", sep=",", names= ["user", "item", "rating", "genre"]) # 
# user_recommendation = pd.read_csv(f"goodbook/results/RS-{topk}_{p_obfuscation}_{preprocess}.csv")
user_recommendation = pd.read_csv(f"goodbook/results/RS-KNN-10_0.01_2Step.csv")


In [181]:
import ast

# Define a function to extract numeric values from dictionary-like strings and replace 'Inf' with 0
def extract_numeric_value(d):
    try:
        # Convert the string representation of dictionary to a Python dictionary
        d = ast.literal_eval(d)
        # Return the numeric value if the dictionary is not empty
        if d:
            return next(iter(d.values()))
        else:
            return np.nan  # Return NaN if the dictionary is empty
    except (SyntaxError, ValueError):
        return np.nan  # Return NaN if there's an error parsing the string

# Apply the function to extract numeric values from the 'KL_div' column and replace 'Inf' with 0
results['KL_div_numeric'] = results['KL_div'].apply(extract_numeric_value).astype(float)

# Replace 'Inf' values with 0
results['KL_div_numeric'].replace(np.inf, 0, inplace=True)

# Apply the function to extract numeric values from the 'JS_div' column and replace 'Inf' with 0
results['JS_div_numeric'] = results['JS_div'].apply(extract_numeric_value).astype(float)

# Replace 'Inf' values with 0
results['JS_div_numeric'].replace(np.inf, 0, inplace=True)

# Group the DataFrame by 'Algorithm' and calculate average KL and JS divergences for each group
average_divergences = results.groupby('Algorithm')[['KL_div_numeric', 'JS_div_numeric']].mean()

# Print the average divergences by algorithm
print("Average KL Divergence by Algorithm:")
print(average_divergences['KL_div_numeric'])
print("\nAverage JS Divergence by Algorithm:")
print(average_divergences['JS_div_numeric'])


Average KL Divergence by Algorithm:
Algorithm
ItemKNN    0.174676
UserKNN    0.182720
Name: KL_div_numeric, dtype: float64

Average JS Divergence by Algorithm:
Algorithm
ItemKNN    0.291609
UserKNN    0.296795
Name: JS_div_numeric, dtype: float64


In [182]:
user_recommendation

,item,score,user,rank,Algorithm
0,75,1.170282,1,1,ItemKNN
1,26,1.125025,1,2,ItemKNN
2,158,1.045481,1,3,ItemKNN
3,62,1.044438,1,4,ItemKNN
4,80,0.921890,1,5,ItemKNN
...,...,...,...,...,...
18855,126,6.032333,943,6,UserKNN
18856,125,5.892788,943,7,UserKNN
18857,59,5.845383,943,8,UserKNN
18858,70,5.733238,943,9,UserKNN


# Coverage

In [183]:

recommendations_dict = {algo: [] for algo in user_recommendation["Algorithm"].unique()}

# Iterate over each row in user_recommendation DataFrame
for index, row in user_recommendation.iterrows():
    # Append the item to the list corresponding to its algorithm
    recommendations_dict[row['Algorithm']].append(row['item'])

# Compute average coverage for each algorithm
average_coverage_per_algorithm = {}

for algo, items in recommendations_dict.items():
    # Compute item coverage for each algorithm
    coverage = len(set(items))
    # Store the average coverage for the algorithm
    average_coverage_per_algorithm[algo] = coverage

# Print average item coverage for each algorithm
for algo, coverage in average_coverage_per_algorithm.items():
    print(f"Algorithm: {algo}, Average Item Coverage: {coverage}")


Algorithm: ItemKNN, Average Item Coverage: 665
Algorithm: UserKNN, Average Item Coverage: 340


## Gini Index

In [184]:
from collections import defaultdict

# Define the GiniIndex class
class GiniIndex:
    """
    Gini Index metric
    """

    def __init__(self, user_recommendation, cutoff=5):
        """
        Constructor
        :param user_recommendation: DataFrame containing recommendation results
        :param cutoff: Number of top recommendations to consider
        """
        self.user_recommendation = user_recommendation
        self.cutoff = cutoff
        self.item_count_per_algo = defaultdict(lambda: defaultdict(int))
        self.free_norm_per_algo = defaultdict(int)
        self.num_items_per_algo = defaultdict(int)

    @staticmethod
    def name():
        """
        Metric Name Getter
        :return: returns the public name of the metric
        """
        return "GiniIndex"

    def user_gini(self, user_recommendations):
        """
        Per User Gini Index
        :param user_recommendations: DataFrame containing user recommendations
        """
        for _, row in user_recommendations.iterrows():
            algo = row['Algorithm']
            item = row['item']
            self.free_norm_per_algo[algo] += 1
            self.item_count_per_algo[algo][item] += 1

    def eval(self):
        """
        Evaluation function
        :return: a dictionary containing the Gini Index for each algorithm
        """
        for algo, algo_recommendations in self.user_recommendation.groupby('Algorithm'):
            self.num_items_per_algo[algo] = len(set(algo_recommendations['item']))
            self.user_gini(algo_recommendations)

        gini_index_per_algo = {}
        for algo in self.item_count_per_algo:
            sorted_item_count = sorted(self.item_count_per_algo[algo].values())
            gini = sum([(2 * (j + (self.num_items_per_algo[algo] - len(sorted_item_count)) + 1) - self.num_items_per_algo[algo] - 1) * (cs / self.free_norm_per_algo[algo]) for j, cs in enumerate(sorted_item_count)]) / (self.num_items_per_algo[algo] - 1)
            gini = 1 - gini
            gini_index_per_algo[algo] = gini
            

        return gini_index_per_algo

# Load your user_recommendations DataFrame

# Initialize the GiniIndex object
gini_index_metric = GiniIndex(user_recommendation)

# Calculate the Gini Index per algorithm
gini_index_per_algo = gini_index_metric.eval()

# Print the results
print("Gini Index per Algorithm:")
for algo, gini_index in gini_index_per_algo.items():
    print(f"{algo}: {gini_index}")


Gini Index per Algorithm:
ItemKNN: 0.35028906719135267
UserKNN: 0.26071190608019956
